# How to get the analysis at data points
This notebook show we can obtain the values of the gridded field at the locations of the observations.

In [ ]:
using DIVAnd
using Makie, CairoMakie, GeoMakie
using Dates
using Statistics
using Random
include("../config.jl")

## Data reading
Download the data file `Salinity.bigfile` (if it is not already present). It will be placed in the same directory as this notebook. This file is a space-separated file containing data based on the [World Ocean Database](https://www.nodc.noaa.gov/OC5/WOD/pr_wod.html).

In [ ]:
fname = salinitybigfile
download_check(salinitybigfile, salinitybigfileURL)

Load the data

In [ ]:
obsval,obslon,obslat,obsdepth,obstime,obsid = loadbigfile(fname);

In [ ]:
# surface values for the month January
sel = (obsdepth .< 10)
x = (obslon[sel], obslat[sel]);
v = obsval[sel] .- mean(obsval[sel]);
@info("$(length(findall(sel))) selected data points")

We create a grid around the Black Sea.

In [ ]:
dx = dy = 0.1
lonr = 27:dx:42.
latr = 40:dy:47.5
mask,(pm,pn),(xi,yi) = DIVAnd_rectdom(lonr,latr);

In [ ]:
bathname = gebco04file
download_check(gebco04file, gebco04fileURL)

In [ ]:
bx,by,b = load_bath(bathname,true,lonr,latr);

In [ ]:
mask = falses(size(b,1),size(b,2))

for j = 1:size(b,2)
    for i = 1:size(b,1)
        mask[i,j] = b[i,j] >=1.0
    end
end

## Make the analysis

In [ ]:
len=0.3
epsilon2=1.0;
fi, s = DIVAndrun(mask, (pm,pn), (xi,yi), x , v, len, epsilon2);

### Analysis plot + data

In [ ]:
Smin = 5.
Smax = 22.

In [ ]:
fig = Figure()
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", title = "Gridded field and observations", xticks=25:3.:42, yticks=40.:1.:48)
hm = heatmap!(ga, lonr, latr, fi .+ mean(obsval[sel]), interpolate = false, colorrange=(Smin, Smax))
sc = scatter!(ga, obslon[sel], obslat[sel], color=obsval[sel], markersize=5, colorrange=(Smin, Smax))
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
contour!(ga, bx, by, b, levels=[0.], color=:black)
Colorbar(fig[1, 2], hm)
fig

## Compute residuals
The function `DIVAnd_residualobs` computes the residuals at the real data location, meaning that the artificial data points possibly used with the constrains are not taken into account.

In [ ]:
dataresidual = DIVAnd_residualobs(s, fi);

In [ ]:
fig = Figure()
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", title = "Residuals", xticks=25:3.:42, yticks=40.:1.:48)
sc = scatter!(ga, obslon[sel], obslat[sel], color=dataresidual, markersize=5, colorrange=(-5, 5), colormap=Reverse(:RdBu))
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
contour!(ga, bx, by, b, levels=[0.], color=:black)
Colorbar(fig[1, 2], sc)
fig

## Analysis at data points

The residuals are defined as the difference between the observations and the analysis at the observation locations, hence the analysis at data points can be recovered as:

In [ ]:
fi_obs = obsval[sel] - dataresidual;
@show size(fi_obs);

### Plot obs vs. analysis at data points
They look similar, we take a tighter interval for the colorbar to make the difference more visible.

In [ ]:
fig = Figure(size=(900, 400))
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", title = "Analysis at data points", xticks=25:3.:42, yticks=40.:1.:48)
sc1 = scatter!(ga, obslon[sel], obslat[sel], color=fi_obs, markersize=5, colorrange=(12, 20.))
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
contour!(ga, bx, by, b, levels=[0.], color=:black)
Colorbar(fig[1, 2], sc1)

ga2 = GeoAxis(fig[1, 3];  dest = "+proj=merc", title = "Original observations", xticks=25:3.:42, yticks=40.:1.:48)
sc2 = scatter!(ga2, obslon[sel], obslat[sel], color=obsval[sel], markersize=5, colorrange=(12, 20.))
xlims!(ga2, (bx[1], bx[end]))
ylims!(ga2, (by[1], by[end]))
contour!(ga2, bx, by, b, levels=[0.], color=:black)
Colorbar(fig[1, 4], sc2)
fig